<a href="https://colab.research.google.com/github/akankshanehete/RNNtesting/blob/main/RNN_binarytesting_gaussian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization,Masking
from scipy import stats

In [ ]:
#%% List of labels for each column of the dataframe

columns = ['Headx','Heady', 'Headz','LWrsx','LWrsy', 'LWrsz','LElbx','LElby','LElbz','RWrsx','RWrsy','RWrsz','RElbx','RElby','RElbz','LAnkx','LAnky','LAnkz','LKnex','LKney','LKnez','RAnkx','RAnky','RAnkz','RKnex','RKney','RKnez']
columnswtime = ['Time', 'Headx','Heady', 'Headz','LWrsx','LWrsy', 'LWrsz','LElbx','LElby','LElbz','RWrsx','RWrsy','RWrsz','RElbx','RElby','RElbz','LAnkx','LAnky','LAnkz','LKnex','LKney','LKnez','RAnkx','RAnky','RAnkz','RKnex','RKney','RKnez']
x_columns = ['Headx', 'LWrsx', 'LElbx', 'RWrsx', 'RElbx','LAnkx','RAnkx', 'LKnex','RKnex']
y_columns = ['Heady', 'LWrsy', 'LElby', 'RWrsy', 'RElby','LAnky','RAnky', 'LKney','RKney']
z_columns = ['Headz', 'LWrsz', 'LElbz', 'RWrsz', 'RElbz','LAnkz','RAnkz', 'LKnez','RKnez']

In [ ]:
drive.mount('/content/gdrive')
path='/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/sub1/normal/Clapping.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Just needed in case you'd like to append it to an array
filenames = []
binaryclasses = []
actionclasses = []
subjectids = []

for dirname in os.listdir('/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset'):
  if not(dirname.endswith('Store')):
    for dir1name in os.listdir('/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/'+dirname):
      for filename in os.listdir('/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/'+dirname +'/'+dir1name):
        if filename.endswith("csv"): 
        # Your code comes here such as 
          filenames.append('/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/'+dirname +'/'+dir1name+'/'+ filename)
          binaryclasses.append(dir1name)
          subjectids.append(dirname[3:])
          actionclasses.append(filename[:-4])

print(filenames)
print(binaryclasses)
print(actionclasses)
print(subjectids)

['/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/sub10/aggressive/Punching.csv', '/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/sub10/aggressive/Kneeing.csv', '/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/sub10/aggressive/Elbowing.csv', '/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/sub10/aggressive/Pushing.csv', '/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/sub10/aggressive/Frontkicking.csv', '/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/sub10/aggressive/Sidekicking.csv', '/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/sub10/aggressive/Pulling.csv', '/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/sub10/aggressive/Headering.csv', '/content/gdrive/My Drive/Colab Notebooks/Sensor_Datasets_RNNs/sensordataset/sub10/aggressive/Slapping.csv', '/content/gdr

In [ ]:
sensordataframes = []


for file in filenames:
  df = pd.read_csv(file)
  df[columns] = df[columns].replace(to_replace = 0,  method='ffill')
  sensordataframes.append(df)


In [ ]:
# speed and acceleration columns in the dataset
speed_columns = ['Speed_Head', 'Speed_LWrs', 'Speed_LElb', 'Speed_RWrs', 'Speed_RElb', 'Speed_LAnk', 'Speed_RAnk', 'Speed_LKne', 'Speed_RKne']
acc_columns = ['Acc_Head', 'Acc_LWrs', 'Acc_LElb', 'Acc_RWrs', 'Acc_RElb', 'Acc_LAnk', 'Acc_RAnk', 'Acc_LKne', 'Acc_RKne']
velocity_columns = ['Velo_Head', 'Velo_LWrs', 'Velo_LElb', 'Velo_RWrs', 'Velo_RElb', 'Velo_LAnk', 'Velo_RAnk', 'Velo_LKne', 'Velo_RKne']


In [ ]:
# adding target labels to dataframe for subjectid, action and normal/aggressive
actions_20_class = ['Punching', 'Kneeing', 'Elbowing', 'Pushing', 'Frontkicking', 'Sidekicking', 'Pulling', 'Headering', 'Slapping', 'Hamering', 'Clapping', 'Jumping', 'Running', 'Seating', 'Handshaking', 'Standing', 'Hugging', 'Walking', 'Bowing', 'Waving']
actions_8_class = ['Punching', 'Kneeing', 'Elbowing', 'Frontkicking', 'Clapping', 'Running', 'Seating', 'Walking']
actions_aggressive = ['Punching', 'Kneeing', 'Elbowing', 'Pushing', 'Frontkicking', 'Sidekicking', 'Pulling', 'Headering', 'Slapping', 'Hamering']
actions_normal = ['Clapping', 'Jumping', 'Running', 'Seating', 'Handshaking', 'Standing', 'Hugging', 'Walking', 'Bowing', 'Waving']
actions_16_class = ['Punching', 'Kneeing', 'Elbowing', 'Frontkicking', 'Pulling', 'Headering', 'Clapping', 'Running', 'Seating', 'Walking', 'Bowing', 'Handshaking', 'Slapping','Kneeing','Bowing','Standing']
actions_6_class = ['Punching', 'Kneeing', 'Elbowing','Seating', 'Walking','Clapping']

# choose the classes to test
actionlist = actions_8_class
num_classes = len(actionlist)

# choose the window size
window_size = 250

# choose how many timestamps we want to remove from the starting of each dataset
n_timestamps_remove = 200

# choose what percentage of data want for testing
testing_percent = 0.5

for i in range (len(sensordataframes)):
  sensordataframes[i]['subjectid'] = subjectids[i]
  sensordataframes[i]['actiontarget'] = actionclasses[i]

  for action in actionlist: 
    if actionclasses[i] == action: 
      sensordataframes[i]['actiontarget'] = int(actionlist.index(action))

  if binaryclasses[i] == 'aggressive':
    sensordataframes[i]['classtarget'] = 1
  else:
    sensordataframes[i]['classtarget'] = 0
  
  #sensordataframes[i] = sensordataframes[i].dropna()
  
sensordataframes[0].columns

Index(['Unnamed: 0', 'Time', 'Headx', 'Heady', 'Headz', 'LWrsx', 'LWrsy',
       'LWrsz', 'LElbx', 'LElby', 'LElbz', 'RWrsx', 'RWrsy', 'RWrsz', 'RElbx',
       'RElby', 'RElbz', 'LAnkx', 'LAnky', 'LAnkz', 'LKnex', 'LKney', 'LKnez',
       'RAnkx', 'RAnky', 'RAnkz', 'RKnex', 'RKney', 'RKnez', 'subjectid',
       'actiontarget', 'classtarget'],
      dtype='object')

In [ ]:
# remove the first n timestamps from all dataframes
for sensordf in sensordataframes: 
  sensordf = sensordf.drop(index=df.index[:n_timestamps_remove])

In [ ]:
# adding velocity columns to all of the sensordataframes
# adding velocity columns to all of the sensordataframes
speed_columns = ['Velo_Head', 'Velo_LWrs', 'Velo_LElb', 'Velo_RWrs', 'Velo_RElb', 'Velo_LAnk', 'Velo_RAnk', 'Velo_LKne', 'Velo_RKne']
speed_columnswtarget = ['Velo_Head', 'Velo_LWrs', 'Velo_LElb', 'Velo_RWrs', 'Velo_RElb', 'Velo_LAnk', 'Velo_RAnk', 'Velo_LKne', 'Velo_RKne','actiontarget', 'classtarget']
# for speed_df in sensordataframes:
#   for i in range(0, len(x_columns)):
#     xyz = speed_df[[x_columns[i], y_columns[i], z_columns[i]]]
#     speed_df['Velo_'+ x_columns[i][:-1]] = np.linalg.norm(xyz.values,axis=1)

for speed_df in sensordataframes:
  # jj: edit for finding diff.
  for j in range(len(columns)):
    # speed_df['shift_'+columns[j]] = (speed_df[columns[j]]- speed_df[columns[j]].shift()).fillna(0)
    speed_df['diff_'+columns[j]] = speed_df[columns[j]].diff()
  
  for i in range(0, len(x_columns)):
    vec = speed_df[['diff_'+x_columns[i], 'diff_'+y_columns[i], 'diff_'+z_columns[i]]]
    speed_df['Velo_'+x_columns[i][:-1]] = np.linalg.norm(vec.values,axis=1)

print(sensordataframes[0][speed_columns])


      Velo_Head  Velo_LWrs  Velo_LElb   Velo_RWrs   Velo_RElb   Velo_LAnk  \
0           NaN        NaN        NaN         NaN         NaN         NaN   
1      0.000000   0.000000   0.000000    0.000000    0.000000    0.000000   
2      0.055299   0.000000   0.000000    0.000000    0.000000    0.000000   
3      0.000000   0.148674   0.053572    0.000000    0.000000    0.000000   
4      0.000000   0.000000   0.000000    0.000000    0.000000    0.165578   
...         ...        ...        ...         ...         ...         ...   
1854   0.000000   0.000000   0.000000    0.000000    0.000000  187.744919   
1855   0.000000   0.000000   0.000000  568.493196  592.363697    0.000000   
1856   0.000000   0.000000   0.000000    0.000000    0.000000    0.000000   
1857        NaN        NaN        NaN         NaN         NaN         NaN   
1858        NaN        NaN        NaN         NaN         NaN         NaN   

       Velo_RAnk   Velo_LKne   Velo_RKne  
0            NaN         NaN    

In [ ]:
# sorting all sensor dataframes into required number of classes (eg. 8, 16, 20)
a =[]
actiontargets = []
classtargets = []
subjectids = []
for sensordf in sensordataframes:
  if type(sensordf['actiontarget'][1]) != str:
    a.append(sensordf)
    actiontargets.append(sensordf['actiontarget'][1])
    classtargets.append(sensordf['classtarget'][1])
    subjectids.append(sensordf['subjectid'][1])
 
sensordataframes = a
print(len(sensordataframes))
print(classtargets)
print(subjectids)
print(actiontargets)



80
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1]
['10', '10', '10', '10', '10', '10', '10', '10', '3', '3', '3', '3', '3', '3', '3', '3', '5', '5', '5', '5', '5', '5', '5', '5', '6', '6', '6', '6', '6', '6', '6', '6', '8', '8', '8', '8', '8', '8', '8', '8', '9', '9', '9', '9', '9', '9', '9', '9', '7', '7', '7', '7', '7', '7', '7', '7', '4', '4', '4', '4', '4', '4', '4', '4', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2']
[0, 1, 2, 3, 4, 5, 6, 7, 4, 6, 5, 7, 0, 1, 3, 2, 1, 0, 3, 2, 4, 5, 6, 7, 4, 5, 6, 7, 1, 0, 3, 2, 5, 4, 7, 6, 1, 0, 3, 2, 0, 1, 3, 2, 4, 5, 6, 7, 5, 7, 4, 6, 0, 1, 3, 2, 1, 0, 3, 2, 5, 6, 4, 7, 6, 4, 5, 7, 0, 1, 2, 3, 5, 4, 7, 6, 0, 1, 3, 2]


In [ ]:
trainingdfs = []
traininglabels = []

testingdfs=[]
testinglabels=[]

# # scenario 3) Subject Imbalance
# for i in range(len(sensordataframes)):
#   if int(subjectids[i]) >= 5:
#     testingdfs.append(sensordataframes[i])
#     testinglabels.append(classtargets[i])
#   else:
#     if classtargets[i] == 0: 
#       trainingdfs.append(sensordataframes[i])
#       traininglabels.append(classtargets[i])
#     elif classtargets[i] == 1 and int(subjectids[i]) == 1:
#       trainingdfs.append(sensordataframes[i])
#       traininglabels.append(classtargets[i])

# # scenario 2) Class Imbalance
# for i in range(len(sensordataframes)):
#   if int(subjectids[i]) >= 5:
#     testingdfs.append(sensordataframes[i])
#     testinglabels.append(classtargets[i])
#   else:
#     if classtargets[i] == 1: 
#       trainingdfs.append(sensordataframes[i])
#       traininglabels.append(classtargets[i])
#     elif classtargets[i] == 0 and int(actiontargets[i]) == 5:
#       trainingdfs.append(sensordataframes[i])
#       traininglabels.append(classtargets[i])

# scenario 1) Random Choose
j = 0
for i in range(len(sensordataframes)):
  if int(subjectids[i]) >= 5:
    testingdfs.append(sensordataframes[i])
    testinglabels.append(classtargets[i])
  else:
    if classtargets[i] == 0: 
      trainingdfs.append(sensordataframes[i])
      traininglabels.append(classtargets[i])
    elif classtargets[i] == 1 and j <= 3:
      trainingdfs.append(sensordataframes[i])
      traininglabels.append(classtargets[i])
      j += 1

print(traininglabels)
print(testinglabels)

[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


In [ ]:
# create new dataframes containing the rolling means and standard deviations
gaussian_velo = ['Velo_Head_avg','Velo_Head_std', 'Velo_LWrs_avg','Velo_LWrs_std', 'Velo_LElb_avg','Velo_LElb_std', 
                    'Velo_RWrs_avg', 'Velo_RWrs_std','Velo_RElb_avg','Velo_RElb_std', 'Velo_LAnk_avg', 'Velo_LAnk_avg',
                    'Velo_RAnk_avg','Velo_RAnk_std', 'Velo_LKne_avg', 'Velo_LKne_std','Velo_RKne_avg','Velo_RKne_std']

# function for creating new dataframe that contains means and standard deviation from sensordataframes
# period means window length ( e.g. 250 for a window length of 250 timestamps)

def extract_gaussian_windowing(sensordf, period): 
  windowdf = pd.DataFrame(columns=gaussian_velo)
  for column in velocity_columns: 
    # using the mean and variance for the windowing approach
    windowdf[column+'_avg'] = sensordf[column].groupby(np.arange(len(sensordf))//period).mean()
    windowdf[column+'_std'] = sensordf[column].groupby(np.arange(len(sensordf))//period).var()
  windowdf.drop(columns=['Velo_LAnk_avg'])
  return windowdf.dropna() 




In [ ]:
# creating a list to hold all the windowing dataframes
gaussian_win_dfs = []

testing_gaussian_dfs = []
training_gaussian_dfs = []

# creating a list to hold the lengths of the dataframes after they have been windowed and the mean and std of each window has been calcualted
training_gaussian_df_lens = []
testing_gaussian_df_lens = []

# creating a series of dataframes with the window size

for df in trainingdfs: 
  gaussian = extract_gaussian_windowing(df, window_size)
  training_gaussian_dfs.append(gaussian)
  training_gaussian_df_lens.append(len(gaussian))

for df in testingdfs: 
  gaussian = extract_gaussian_windowing(df, window_size)
  testing_gaussian_dfs.append(gaussian)
  testing_gaussian_df_lens.append(len(gaussian))

pd.Series(testing_gaussian_df_lens).describe()
pd.Series(training_gaussian_df_lens).describe()

count    20.000000
mean     11.300000
std       3.213459
min       7.000000
25%       8.750000
50%      11.500000
75%      14.250000
max      16.000000
dtype: float64

In [ ]:

# creating sequences from the windowing dataframes to feed into the RNN
training_sequences = list()
testing_sequences = list()


for i in range(len(testing_gaussian_dfs)):
  values = testing_gaussian_dfs[i][gaussian_velo].values
  testing_sequences.append(values)

for i in range(len(training_gaussian_dfs)):
  values = training_gaussian_dfs[i][gaussian_velo].values
  training_sequences.append(values)


features= 20



In [ ]:
# padding each sequence with a certain value in order to make them all the same length because RNN does not accept variable sequence lengths
to_pad = 200
new_seq = []
for one_seq in training_sequences:
    len_one_seq = len(one_seq)
    # -1 is the special value that sequences are padded with that the RNN can mask
    last_val = -1
    n = to_pad - len_one_seq
   
    to_concat = an_array = np.full((n, len(one_seq[0])), -1)
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)


# truncate each sequence to the desired length (e.g how many windows in each training sequence?)
from keras.preprocessing import sequence
seq_len = 30
final_seq=sequence.pad_sequences(new_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

training_sequences = np.array(final_seq)
print(training_sequences[0])



[[ 2.66961303e-01  7.15126791e+00  1.64418941e+00  1.77755802e+02
   2.85040130e+00  5.30963770e+02  1.83976649e+00  2.33123120e+02
   3.40103622e+00  5.26805096e+02  9.01453341e-02  9.01453341e-02
   9.01453341e-02  9.01453341e-02  9.45890218e-02  5.07565763e-01
   2.56535050e-02  9.72542146e-03  3.06195270e-02  2.35650996e-02]
 [ 5.32455646e-02  5.23860982e-02  6.21506745e-01  1.15787768e+01
   1.32136019e+00  4.62585750e+01  7.15344297e-01  1.21595898e+01
   1.27375299e+00  3.69666996e+01  3.16498628e-02  3.16498628e-02
   3.16498628e-02  3.16498628e-02  5.42182770e-02  3.14291912e-02
   1.82162208e-02  2.08774428e-03  1.74485253e-02  1.62275599e-03]
 [ 1.20068285e-01  3.67952497e-01  5.33134865e-01  6.68427213e+00
   1.10995144e+00  3.13796270e+01  6.39841165e-01  9.37590915e+00
   1.25759549e+00  4.12206238e+01  3.02228006e-02  3.02228006e-02
   3.02228006e-02  3.02228006e-02  5.64806380e-02  1.06328881e-01
   1.67590328e-02  1.54589440e-03  1.96161470e-02  4.64247267e-03]
 [ 5.97

In [ ]:
# padding each sequence with a certain value in order to make them all the same length because RNN does not accept variable sequence lengths
to_pad = 200
new_seq = []
for one_seq in testing_sequences:
    len_one_seq = len(one_seq)
    # -1 is the special value that sequences are padded with that the RNN can mask
    last_val = -1
    n = to_pad - len_one_seq
   
    to_concat = an_array = np.full((n, len(one_seq[0])), -1)
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)


# truncate each sequence to the desired length (e.g how many windows in each training sequence?)
from keras.preprocessing import sequence
seq_len = 30
final_seq=sequence.pad_sequences(new_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

testing_sequences = np.array(final_seq)
print(testing_sequences)

[[[ 5.03858855e+00  1.96740387e+03  5.45985768e+00 ...  3.75930185e+03
    1.87147602e+00  2.60664255e+02]
  [ 3.23168015e+00  4.26160866e+02  5.97631900e+00 ...  1.06617293e+02
    1.77296074e+00  8.60299037e+01]
  [ 4.62978509e+00  8.76946506e+02  6.70579301e+00 ...  4.48260728e+01
    2.02015622e+00  1.17574979e+02]
  ...
  [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00 ... -1.00000000e+00
   -1.00000000e+00 -1.00000000e+00]
  [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00 ... -1.00000000e+00
   -1.00000000e+00 -1.00000000e+00]
  [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00 ... -1.00000000e+00
   -1.00000000e+00 -1.00000000e+00]]

 [[ 2.08832323e+00  3.72093604e+02  3.66069857e+00 ...  8.68978628e+01
    7.03423245e-02  4.70959209e-02]
  [ 3.43666061e+00  2.71493027e+02  4.12042022e+00 ...  1.43499977e+03
    5.84883416e+00  2.27473556e+03]
  [ 2.18601656e+00  1.36723307e+02  4.55032700e+00 ...  5.35399681e+01
    6.80790708e+00  6.89087373e+02]
  ...
  [-1.00000000e+00 -1.0

In [ ]:
x_train = np.array(training_sequences)
y_train = np.array(traininglabels)
x_test = np.array(testing_sequences)
y_test = np.array(testinglabels)

In [ ]:
# model constants
BATCH_SIZE = 100
EPOCHS = 10

In [ ]:
# building the rnn model (adding layers)

accuracies = []
for _ in range (10):
  model = Sequential()
  # add masking layer to the RNN to avoid the padded values in series since they are of variable length
  model.add(tf.keras.layers.Masking(mask_value=-1, input_shape=(seq_len, features)))
  model.add(LSTM(128, input_shape=(x_train.shape[1:]), return_sequences=True))

  model.add(Dropout(0.2))
  model.add(BatchNormalization())  

  model.add(LSTM(128, return_sequences=True))
  model.add(Dropout(0.1))
  model.add(BatchNormalization())

  model.add(LSTM(128))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.2))

  # last layer has 2 nodes due to there being binary classification
  model.add(Dense(2, activation='softmax'))

  opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
    
  model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'])


  # training the model
  model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_test,y_test))
  result = model.evaluate(x_test,y_test)
  accuracies.append(result[1])

mean_accuracy = sum(accuracies)/len(accuracies)
print('mean accuracy: ' + str(mean_accuracy))
print('standard deviation:' + str(np.std(accuracies)))




Epoch 1/10
1/1 [==============================] - 20s 20s/step - loss: 0.9692 - accuracy: 0.5500 - val_loss: 0.6997 - val_accuracy: 0.5208
Epoch 2/10
1/1 [==============================] - 0s 192ms/step - loss: 0.3649 - accuracy: 0.8000 - val_loss: 0.6891 - val_accuracy: 0.5417
Epoch 3/10
1/1 [==============================] - 0s 188ms/step - loss: 0.1678 - accuracy: 1.0000 - val_loss: 0.6797 - val_accuracy: 0.5833
Epoch 4/10
1/1 [==============================] - 0s 180ms/step - loss: 0.1689 - accuracy: 0.9500 - val_loss: 0.6698 - val_accuracy: 0.6250
Epoch 5/10
1/1 [==============================] - 0s 183ms/step - loss: 0.1526 - accuracy: 0.9500 - val_loss: 0.6625 - val_accuracy: 0.6458
Epoch 6/10
1/1 [==============================] - 0s 187ms/step - loss: 0.0942 - accuracy: 0.9500 - val_loss: 0.6552 - val_accuracy: 0.6875
Epoch 7/10
1/1 [==============================] - 0s 180ms/step - loss: 0.0238 - accuracy: 1.0000 - val_loss: 0.6486 - val_accuracy: 0.7083
Epoch 8/10
1/1 [=====